Given a set of basic statistical results, which give coefficients for individual transitions, we search for coefficients which are different than mean of all other coefficients. We do this for each coefficient and record the p-values.

This will save results in a format that is conducive for working with existing plotting code. 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import multiprocessing as mp
from pathlib import Path
import pickle
import re

import numpy as np

from keller_zlatic_vnc.whole_brain.whole_brain_stat_functions import test_for_different_than_avg_beta

## Parameters go here

In [3]:
ps = dict()
ps['save_folder'] = r'\\dm11\bishoplab\projects\keller_vnc\results\single_subject_spont_stats'
ps['basic_rs_file'] = 'spont_1_5_5_long_bl_co_4_start_locked_0_6_ece_pre_tc_post_tc.pkl'

## Load the basic results

In [4]:
with open(Path(ps['save_folder']) / ps['basic_rs_file'], 'rb') as f:
    basic_rs = pickle.load(f)

In [5]:
beh_trans = basic_rs['beh_trans']

## Search for values different than "other" mean

In [6]:
def test_for_diff_than_mean_vls(stats, beh_trans, mn_th = 1e-10):
    """ This is a helper function which calculates post-hoc statistics for each group.
    
    A group are all transitions that start with the same behavior. 
    
    For a coefficient in each group, we calculate the p-value that it's value is not larger than the mean of all
    other coefficients in the group. 
    
    If there is only one transition in a group (e.g., for a given start behavior, we only have transitions into
    a single end behavior, we also set the p-value of these coefficients to 1.)
    
    We return all p-values in a single vector, for ease of integration with plotting code, but it should be remembered
    that coefficinets were compared within groups. 
    
    Note: Before computing any stats, this function first makes sure there is a large enough numerical diference between
    the coefficients for the individual behaviors.  If there is not, then beta is set to 0 for all behaviors and p values of 1 
    are returned.  We do this to avoid issues that might arise with limited floating point precision when measuring very
    small differences between means. If the differences are small enough that floating point issues become a concern, then
    they are not of interest to us anyways, so we lose nothing by doing this.  We determine if numerical issues may be a concern
    by fist computing the average of all coefficients and checking if all coefficients are within mn_th of this mean. If this
    is the case, we determine the values are too near one another. 
    
    """
    
    n_coefs = len(beh_trans)
    p_vls = np.zeros(n_coefs)
    beta = np.zeros(n_coefs)
    
    unique_grp_behs = set([t[0] for t in beh_trans])
    
    # Do a quick check to see that mean values for each behavior were different enough to even warrnat doing 
    # stats.  If values were too close, we are going to run into floating points issues, and if the differences
    # were that small anyway, we lose nothing by not checking for differences
    mn_diffs = np.abs(stats['beta'] - np.mean(stats['beta']))
    if np.all(mn_diffs < mn_th):
        new_stats = dict()
        new_stats['beta'] = beta
        new_stats['eq_mean_p'] = np.ones(n_coefs)
        return new_stats
    
    # Process results for each group
    for grp_b in unique_grp_behs:
        keep_cols = np.asarray(np.argwhere([1 if b[0] == grp_b else 0 for b in beh_trans])).squeeze()

        p_vls[keep_cols] = 1 # Initially set all p-values to this group to 1, we will set the p-value 
                             # for the largest coefficient in the code below, but do denote that the 
                             # coefficients which are not largest are not to be considered, we set their
                             # p-values to 1. 

        if keep_cols.ndim > 0: # Means we have more than one coefficient
            grp_beta = stats['beta'][keep_cols]
            grp_acm = stats['acm'][np.ix_(keep_cols, keep_cols)]
            if not np.all(np.diag(grp_acm) == np.zeros(grp_acm.shape[0])):
                n_grps = stats['n_grps']
                # Note: alpha below is not important for this function, since we record p-values
                grp_p_vls, _  = test_for_different_than_avg_beta(beta=grp_beta, acm=grp_acm, n_grps=n_grps, alpha=.05)
                p_vls[keep_cols] = grp_p_vls
            
                n_grp_coefs = len(grp_beta)
                new_grp_beta = np.zeros(n_grp_coefs)
                for b_i in range(n_grp_coefs):
                    new_grp_beta[b_i] = grp_beta[b_i] - ((np.sum(grp_beta) - grp_beta[b_i])/(n_grp_coefs - 1))
            
                beta[keep_cols] = new_grp_beta
            else:
                pass 
                # We don't need to do anything - because we already set all p_vls for this group to 1
        else:
            pass
            # We don't need to do anything - because we already set all p_vls for this group to 1
    
    
    
    new_stats = dict()
    new_stats['beta'] = beta
    new_stats['eq_mean_p'] = p_vls
    
    return new_stats
    

In [7]:
all_mean_stats = [test_for_diff_than_mean_vls(s, basic_rs['beh_trans']) for s in basic_rs['full_stats']]

KeyboardInterrupt: 

In [10]:
n_cpu = 10
pool = mp.Pool(n_cpu)

88

In [ ]:
def helper_f(s):
    return test_for_diff_than_mean_vls(s, basic_rs['beh_trans'])

In [ ]:
all_mean_stats = pool.map(helper_f, basic_rs['full_stats'][0:10])

## Now save our results

In [ ]:
rs = {'ps': ps, 'full_stats': all_mean_stats, 'beh_trans': basic_rs['beh_trans']}

save_folder = ps['save_folder']
save_name = ps['basic_rs_file'].split('.')[0] + '_mean_cmp_stats.pkl'

save_path = Path(save_folder) / save_name
with open(save_path, 'wb') as f:
    pickle.dump(rs, f)

## Debug code

In [ ]:
save_folder

In [ ]:
save_name

In [11]:
beh_trans

[('B', 'TC'), ('P', 'F'), ('Q', 'B'), ('Q', 'F'), ('Q', 'TC'), ('TC', 'B')]